In [6]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from sklearn.utils.class_weight import compute_class_weight
import os
import numpy as np

# Mount Google Drive (if using Google Colab)
from google.colab import drive
drive.mount('/content/drive')

# Constants
IMG_HEIGHT, IMG_WIDTH = 224, 224
BATCH_SIZE = 32
EPOCHS = 25

# Directories
TRAIN_DIR = "/content/drive/MyDrive/sinusities/waters_view_dataset"
VAL_DIR = "/content/drive/MyDrive/sinusities/waters_view_dataset"

# Data Augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True
)

val_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

# Create train and validation datasets
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

val_generator = val_datagen.flow_from_directory(
    VAL_DIR,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

# Compute class weights to handle imbalance
classes = list(train_generator.class_indices.keys())
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights_dict = {i: weight for i, weight in enumerate(class_weights)}
print("Class Weights:", class_weights_dict)

# Load MobileNetV2 as a feature extractor
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(IMG_HEIGHT, IMG_WIDTH, 3))
base_model.trainable = False  # Freeze the base model

# Add a classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification

model = Model(inputs=base_model.input, outputs=output)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    validation_data=val_generator,
    validation_steps=val_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
    class_weight=class_weights_dict
)

# Save the trained model
model.save('/content/drive/MyDrive/sinusities/improved_waters_view_validator_model.h5')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 1784 images belonging to 2 classes.
Found 1784 images belonging to 2 classes.
Class Weights: {0: 81.0909090909091, 1: 0.503102086858432}
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/25


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


55/55 ━━━━━━━━━━━━━━━━━━━━ 192s 3s/step - accuracy: 0.4119 - loss: 1.2132 - val_accuracy: 0.5977 - val_loss: 0.6541
Epoch 2/25
 1/55 ━━━━━━━━━━━━━━━━━━━━ 1:00 1s/step - accuracy: 0.5625 - loss: 0.4454

/usr/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.5625 - loss: 0.4454 - val_accuracy: 0.5417 - val_loss: 0.6594
Epoch 3/25
55/55 ━━━━━━━━━━━━━━━━━━━━ 199s 3s/step - accuracy: 0.5894 - loss: 1.0975 - val_accuracy: 0.7489 - val_loss: 0.5540
Epoch 4/25
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.5625 - loss: 0.3416 - val_accuracy: 0.5833 - val_loss: 0.7147
Epoch 5/25
55/55 ━━━━━━━━━━━━━━━━━━━━ 203s 3s/step - accuracy: 0.6471 - loss: 0.6277 - val_accuracy: 0.8585 - val_loss: 0.4877
Epoch 6/25
55/55 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.7188 - loss: 0.2656 - val_accuracy: 0.7917 - val_loss: 0.5064
Epoch 7/25
55/55 ━━━━━━━━━━━━━━━━━━━━ 193s 3s/step - accuracy: 0.6782 - loss: 0.6071 - val_accuracy: 0.8960 - val_loss: 0.4547
Epoch 8/25
55/55 ━━━━━━━━━━━━━━━━━━━━ 61s 1s/step - accuracy: 0.7812 - loss: 0.2627 - val_accuracy: 1.0000 - val_loss: 0.3225
Epoch 9/25
55/55 ━━━━━━━━━━━━━━━━━━━━ 186s 3s/step - accuracy: 0.6956 - loss: 1.1014 - val_accuracy: 0.9381 - val_loss: 0.4